In [1]:
import numpy as np 
import pandas as pd 

In [2]:
# load data and check size (T)
with open("sequence.txt", "r") as f:
    data = eval(f.read())

In [37]:
class ModelHMM:
    
    def __init__(self, A, B, data, freeze_B=None): 
        
        # initialize model parameters 
        self.A = A 
        self.B = B
        self.num_hidden, self.num_obs = B.shape
        assert len(freeze_B) == self.num_hidden
        self.freeze_B = freeze_B
        
        # initialize data parameters 
        self.data = data 
        self.T = len(data)
        
        # initialize additional attributes to use to training 
        self.forwards = np.zeros((self.num_hidden, self.T))
        self.backwards = np.zeros((self.num_hidden, self.T))
        self.coefs = np.zeros(self.T) # coefs used to rescale forward/backward probas for numerical stability 
        self.gammas = np.zeros((self.num_hidden, self.T))
        self.ixs = np.zeros((self.num_hidden, self.num_hidden, self.T-1))
        
    def forward_pass(self, pi=None):
        
        # except during first pass when an initial_pi is passed in, approximate pi with gammas 
        if pi is None:
            pi = self.gammas[:,0]
        
        # compute first time step 
        self.forwards[:,0] = np.multiply(pi, self.B[:, self.data[0]-1])
        self.coefs[0] = 1 / self.forwards[:,0].sum()
        self.forwards[:,0] = self.coefs[0] * self.forwards[:,0]

        # iterate over rest of time steps 
        for t in range(1, self.T): 
            self.forwards[:,t] = np.multiply(self.A @ self.forwards[:,t-1], self.B[:, self.data[t]-1])
            self.coefs[t] = 1 / self.forwards[:,t].sum()
            self.forwards[:,t] = self.coefs[t] * self.forwards[:,t]
            
    def backward_pass(self): 
        
        # compute last time step 
        self.backwards[:, -1] = 1
        self.backwards[:, -1] = self.coefs[-1] * self.backwards[:, -1]

        # iterate backward through rest of time steps 
        for t in reversed(range(self.T-1)):
            self.backwards[:,t] = np.multiply(self.A @ self.backwards[:,t+1], self.B[:, self.data[t+1]-1])
            self.backwards[:,t] = self.coefs[t] * self.backwards[:,t]
            
    def e_step(self, pi=None):
        
        self.forward_pass(pi)
        self.backward_pass()
        
        # compute ixs 
        for t in range(self.T-1):
            for i in range(self.num_hidden):
                for j in range(self.num_hidden): 
                    self.ixs[i,j,t] = self.forwards[i,t] * self.A[i,j] * \
                        self.B[j, self.data[t+1]-1] * self.backwards[j,t+1]
        
        # compute gammas 
        for t in range(self.T):
            self.gammas[:,t] = np.multiply(self.forwards[:,t], self.backwards[:,t])
    
    def m_step(self): 
        
        # re-estimate A 
        for i in range(self.num_hidden):
            for j in range(self.num_hidden):
                self.A[i,j] = self.ixs[i,j,:].sum() / self.ixs[i,:,:].sum()        
                
        # re-estimate B 
        for j in range(self.num_hidden):
            if self.freeze_B[j]: # if one die's proba is fixed don't update it 
                pass 
            else:
                for k in range(self.num_obs):
                    obs_mask = np.array([datum == k+1 for datum in self.data])
                    self.B[j,k] = (self.gammas[j,:] * obs_mask).sum() / self.gammas[j,:].sum()
                
    def train(self, num_iter, initial_pi):

        # first step uses initial pi estimate 
        self.e_step(initial_pi)
        self.m_step() 
        print(self.A)
        print(self.B)
        
        # subsequent steps uses gammas to estimate pi 
        for m in range(1, num_iter):
            self.e_step()
            self.m_step() 
            if m % 100 == 0:
                print(m)
                print(self.A)
                print(self.B)    

In [38]:
# two die 
A = np.array([[.49, .51], 
              [.51, .49]])
B = np.array([[1./6, 1./6, 1./6, 1./6, 1./6, 1./6], 
              [3./10, 1./10, 1./10, 1./10, 1./10, 3./10]])
initial_pi = np.array([1., 0.])
freeze_B = [True, False]

model = ModelHMM(A, B, data, freeze_B)
model.train(num_iter=500, initial_pi=initial_pi)
print(model.A)
print(model.B)

[[0.4943581  0.5056419 ]
 [0.50841372 0.49158628]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26124724 0.13904111 0.12677137 0.13431091 0.13592789 0.20270148]]
100
[[0.01869132 0.98130868]
 [0.01433387 0.98566613]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22726301 0.14756106 0.14379993 0.14520211 0.14405796 0.19211593]]
200
[[5.80920331e-04 9.99419080e-01]
 [1.18929686e-03 9.98810703e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26376631 0.14044202 0.1372873  0.13826111 0.13742484 0.18281842]]
300
[[2.77767557e-05 9.99972223e-01]
 [1.31442738e-03 9.98685573e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26958175 0.13930282 0.13622289 0.13712952 0.13635344 0.18140959]]
400
[[1.69918206e-06 9.99998301e-01]
 [1.35547373e-03 9.98644526e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26888864 0.13943867 0.13635    0.13726106 0.13648158

In [39]:
# three die
A = np.array([[.34, .33, .33], 
              [.33, .34, .33],
              [.33, .33, .34]])
B = np.array([[1./6, 1./6, 1./6, 1./6, 1./6, 1./6], 
              [5./10, 1./10, 1./10, 1./10, 1./10, 1./10],
              [1./10, 1./10, 1./10, 1./10, 1./10, 5./10]])
initial_pi = np.array([1., 0., 0.])
freeze_B = [True, False, False]

model = ModelHMM(A, B, data, freeze_B)
model.train(500, initial_pi)
print(model.A)
print(model.B)

[[0.36636581 0.34559279 0.2880414 ]
 [0.34855088 0.36032541 0.29112371]
 [0.3513131  0.34821072 0.30047618]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.34513887 0.15124101 0.14345705 0.15164261 0.15547069 0.05304976]
 [0.07135703 0.17219896 0.15467053 0.16082575 0.15721013 0.2837376 ]]
100
[[0.0150183  0.5817295  0.4032522 ]
 [0.0092998  0.42401765 0.56668256]
 [0.00913962 0.30378152 0.68707886]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.07925248 0.00443427 0.35296344 0.24421655 0.20769626 0.111437  ]
 [0.32001329 0.24719783 0.00594707 0.08251296 0.10110021 0.24322863]]
200
[[2.24395310e-03 9.73142390e-01 2.46136572e-02]
 [7.49982800e-03 6.26222297e-01 3.66277875e-01]
 [4.80452070e-04 3.20222683e-01 6.79296865e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.31080689 0.1113334  0.17358549 0.15241917 0.1577182  0.09413685]
 [0.14287613 0.1838471  0.12034642 0.14289591 0.13559107 0.27444336]]
300
[

In [52]:
import random
fake_data1 = np.random.choice([1,1,1,1,1,2,3,4,5,6], size=2500)
fake_data2 = np.random.choice([1,2,3,4,5,6], size=2500)
fake_data = np.concatenate([fake_data1, fake_data2])
random.shuffle(fake_data)

In [53]:
# two die 
A = np.array([[.49, .51], 
              [.51, .49]])
B = np.array([[1./6, 1./6, 1./6, 1./6, 1./6, 1./6], 
              [3./10, 1./10, 1./10, 1./10, 1./10, 3./10]])
initial_pi = np.array([1., 0.])
freeze_B = [True, False]

model = ModelHMM(A, B, fake_data, freeze_B)
model.train(num_iter=500, initial_pi=initial_pi)
print(model.A)
print(model.B)

[[0.48802061 0.51197939]
 [0.50700558 0.49299442]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.33439031 0.1373213  0.13035525 0.13513153 0.13114394 0.13165767]]
100
[[7.90475956e-05 9.99920952e-01]
 [8.62041187e-04 9.99137959e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.24413439 0.13403725 0.12763764 0.23484941 0.12847611 0.1308652 ]]
200
[[4.48391929e-09 9.99999996e-01]
 [9.15387075e-04 9.99084613e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.24401744 0.13395926 0.12755748 0.23527987 0.12839955 0.1307864 ]]
300
[[2.71846382e-13 1.00000000e+00]
 [9.15417960e-04 9.99084582e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.24401799 0.13395959 0.12755781 0.23527801 0.12839988 0.13078673]]
400
[[1.64813345e-17 1.00000000e+00]
 [9.15417963e-04 9.99084582e-01]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.24401799 0.13395959 0.12755781 0.235

# OLD CODE

In [215]:
class ModelHMM:
    
    def __init__(self, num_hidden, num_obs, data):
        
        # initialize inputs
        self.num_hidden = num_hidden
        self.num_obs = num_obs 
        self.data = data 
        self.T = len(data)
        
        # initialize model parameters (CANNOT BE UNIFORM)
#        self.pi = np.array([[.49, .51]])
        self.A = np.array([[.49, .51], [.51, .49]])
        self.B = np.array([[1./6, 1./6, 1./6, 1./6, 1./6, 1./6], [2./10, 1./10, 1./10, 1./10, 1./10, 2./10]])
#         self.pi = np.full(shape=(self.num_hidden), fill_value=1/self.num_hidden)
#         self.A = np.full(shape=(self.num_hidden, self.num_hidden), fill_value=1/self.num_hidden)
#         self.B = np.full(shape=(self.num_hidden, self.num_obs), fill_value=1/self.num_obs)
        
        # initialize additional attributes to use for training 
        self.forwards = np.zeros((self.num_hidden, self.T))
        self.backwards = np.zeros((self.num_hidden, self.T))
        self.coefs = np.zeros(self.T) # coefs used to rescale forward/backward probas for numerical stability 
        self.gammas = np.zeros((self.num_hidden, self.T))
        self.ixs = np.zeros((self.num_hidden, self.num_hidden, self.T-1))
        
    def forward_pass(self, pi=None):
        
        if pi is None:
            pi = self.gammas[:,0]
        
        # compute first time step 
        self.forwards[:,0] = np.multiply(pi, self.B[:, self.data[0]-1])
        self.coefs[0] = 1 / self.forwards[:,0].sum()
        self.forwards[:,0] = self.coefs[0] * self.forwards[:,0]

        # iterate over rest of time steps 
        for t in range(1, self.T): 
            self.forwards[:,t] = np.multiply(self.A @ self.forwards[:,t-1], self.B[:, self.data[t]-1])
            self.coefs[t] = 1 / self.forwards[:,t].sum()
            self.forwards[:,t] = self.coefs[t] * self.forwards[:,t]
            
    def backward_pass(self): 
        
        # compute last time step 
        self.backwards[:, -1] = 1
        self.backwards[:, -1] = self.coefs[-1] * self.backwards[:, -1]

        # iterate backward through rest of time steps 
        for t in reversed(range(self.T-1)):
            self.backwards[:,t] = np.multiply(self.A @ self.backwards[:,t+1], self.B[:, self.data[t+1]-1])
            self.backwards[:,t] = self.coefs[t] * self.backwards[:,t]
            
    def e_step(self, pi=None):
        
        self.forward_pass(pi)
        self.backward_pass()
        
        # compute ixs 
        for t in range(self.T-1):
            for i in range(self.num_hidden):
                for j in range(self.num_hidden): 
                    self.ixs[i,j,t] = self.forwards[i,t] * self.A[i,j] * \
                        self.B[j, self.data[t+1]-1] * self.backwards[j,t+1]
        
        # compute gammas 
        for t in range(self.T):
            self.gammas[:,t] = np.multiply(self.forwards[:,t], self.backwards[:,t])
    
    def m_step(self): 
        
        # re-estimate A 
        for i in range(num_hidden):
            for j in range(num_hidden):
                self.A[i,j] = self.ixs[i,j,:].sum() / self.ixs[i,:,:].sum()        
                
        # re-estimate B 
        for j in range(num_hidden):
            if j == 0: # assume first die is fair 
                pass 
            else:
                for k in range(num_obs):
                    obs_mask = np.array([datum == k+1 for datum in self.data])
                    self.B[j,k] = (self.gammas[j,:] * obs_mask).sum() / self.gammas[j,:].sum()
                
    def train(self, num_iter):
        initial_pi = np.array([[.7, .3]])
        self.e_step(initial_pi)
        self.m_step() 
        print(self.A)
        print(self.B)
        
        for m in range(1, num_iter):
            print(m)
            self.e_step()
            self.m_step() 
            print(self.A)
            print(self.B)

In [ ]:
# define number of possible states N and number of possible observations
# num_hidden = 2
# num_obs = 6

In [ ]:
# initialize transition and observation probabilities 
# pi = np.full(shape=(num_hidden), fill_value=1/num_hidden)
# A = np.full(shape=(num_hidden,num_hidden), fill_value=1/num_hidden)
# B = np.full(shape=(num_hidden,num_obs), fill_value=1/num_obs)

In [5]:
def forward_pass(A, B, T, data):
    
    # initialize matrix to hold forward probabilities and rescaling factors 
    inter_alpha = np.zeros((num_hidden, T))
    final_alpha = np.zeros((num_hidden, T))
    coef = np.zeros(T)
    
    # compute first time step 
    inter_alpha[:,0] = np.multiply(pi, B[:,data[0]-1])
    coef[0] = 1 / inter_alpha[:,0].sum()
    final_alpha[:,0] = coef[0] * inter_alpha[:,0]
    
    # iterate over rest of time steps 
    for t in range(1, T): 
        inter_alpha[:,t] = np.multiply(A @ final_alpha[:,t-1], B[:,data[t]-1])
        coef[t] = 1 / inter_alpha[:,t].sum()
        final_alpha[:,t] = coef[t] * inter_alpha[:,t]
    
    return final_alpha, coef

In [6]:
# def forward_pass(A, B, T, data):
    
#     # initialize matrix to hold forward probabilities and rescaling factors 
#     alpha = np.zeros((num_hidden, T))
#     coef = np.zeros(T)
    
#     # compute first time step 
#     alpha[:,0] = np.multiply(pi, B[:,data[0]-1])
#     coef[0] = 1 / alpha[:,0].sum()
#     alpha[:,0] = coef[0] * alpha[:,0]
    
#     # iterate over rest of time steps 
#     for t in range(1, T): 
#         alpha[:,t] = np.multiply(A @ alpha[:,t-1], B[:,data[t]-1])
#         coef[t] = 1 / alpha[:,t].sum()
#         alpha[:,t] = coef[t] * alpha[:,t]
    
#     return alpha, coef

In [7]:
# final_alphas, coefs = forward_pass(A, B, T, sample_data)
# print(final_alphas) 
# print(coefs)

In [8]:
# def backward_pass(A, B, T, data, coef): 
    
#     # initialize matrix to hold forward probabilities and rescaling factors 
#     beta = np.zeros((num_hidden, T))

#     # initialize last time step 
#     beta[:, -1] = 1
#     beta[:, -1] = coef[-1] * beta[:, -1]
    
#     # iterate backward through rest of time steps 
#     for t in reversed(range(T-1)):
#         beta[:,t] = np.multiply(A @ beta[:,t+1], B[:,data[t]-1])
#         beta[:,t] = coef[t] * beta[:,t]
        
#     return beta

In [9]:
# def backward_pass(A, B, T, data, coef): 
    
#     # initialize matrix to hold forward probabilities and rescaling factors 
#     inter_beta = np.zeros((num_hidden, T))
#     final_beta = np.zeros((num_hidden, T))

#     # initialize last time step 
#     inter_beta[:, -1] = 1
#     final_beta[:, -1] = coef[-1] * inter_beta[:, -1]
    
#     # iterate backward through rest of time steps 
#     for t in reversed(range(T-1)):
#         inter_beta[:,t] = np.multiply(A @ final_beta[:,t+1], B[:,data[t]-1])
#         final_beta[:,t] = coef[t] * inter_beta[:,t]
        
#     return final_beta

In [10]:
# final_betas = backward_pass(A, B, T, sample_data, coefs)
# final_betas

In [217]:
model = ModelHMM(num_hidden=2, num_obs=6, data=data)
model.train(num_iter=200)
# model.forward_pass()
# model.backward_pass()
# # print(model.backwards)
# model.e_step()
# model.m_step()
# model.A
# model.B
#print(model.gammas)
# print(model.backwards)
# print(model.backwards)
# print(model.forwards)
# print(model.ixs)
# print(model.gammas)
# model.e_step()
# print(model.backwards)

[[0.58153044 0.41846956]
 [0.59854362 0.40145638]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.27031199 0.13426417 0.12408576 0.13016883 0.13049909 0.21067015]]
1
[[0.55966344 0.44033656]
 [0.57471818 0.42528182]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26542583 0.13618153 0.12392392 0.13149707 0.13195921 0.21101244]]
2
[[0.53875706 0.46124294]
 [0.55182181 0.44817819]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26284511 0.13711488 0.12496589 0.13257625 0.13278933 0.20970854]]
3
[[0.51875648 0.48124352]
 [0.52984733 0.47015267]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.26036027 0.13785275 0.12625608 0.13335046 0.13352993 0.2086505 ]]
4
[[0.49961674 0.50038326]
 [0.50877807 0.49122193]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.25802874 0.13846515 0.12745963 0.13428193 0.13415833 0.20760622]]
5
[[0.48129568 0.51870432]
 [0.48858959 0.

[[0.13182087 0.86817913]
 [0.11580842 0.88419158]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22945645 0.14668977 0.14204838 0.14393525 0.14296983 0.19490031]]
44
[[0.12765167 0.87234833]
 [0.1117302  0.8882698 ]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22909504 0.1466758  0.14208301 0.14449433 0.14296538 0.19468644]]
45
[[0.12361007 0.87638993]
 [0.10779455 0.89220545]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22905542 0.14680915 0.14224074 0.1440716  0.14310312 0.19471997]]
46
[[0.11971213 0.88028787]
 [0.10401171 0.89598829]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22870958 0.14678891 0.1422661  0.14463102 0.14309206 0.19451234]]
47
[[0.11593231 0.88406769]
 [0.10035997 0.89964003]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.2286881  0.1469186  0.14241665 0.14419635 0.14322548 0.19455481]]
48
[[0.1122864  0.8877136 ]
 [0.096849

[[0.03391294 0.96608706]
 [0.02570925 0.97429075]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22505905 0.14785398 0.14397639 0.1459529  0.14429651 0.19286118]]
87
[[0.032868   0.967132  ]
 [0.02483779 0.97516221]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22521066 0.14796104 0.14407262 0.1453614  0.14439973 0.19299456]]
88
[[0.03185949 0.96814051]
 [0.02399893 0.97600107]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22498425 0.14787536 0.14401073 0.14598139 0.14432102 0.19282725]]
89
[[0.03087823 0.96912177]
 [0.02318611 0.97681389]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22513991 0.14798246 0.14410631 0.14538426 0.14442407 0.192963  ]]
90
[[0.02993118 0.97006882]
 [0.02240368 0.97759632]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22491464 0.14789521 0.14404262 0.14600808 0.14434379 0.19279566]]
91
[[0.02900965 0.97099035]
 [0.021645

[[0.00888483 0.99111517]
 [0.00590289 0.99409711]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22441782 0.14820427 0.144454   0.14560368 0.14467689 0.19264334]]
130
[[0.0086134  0.9913866 ]
 [0.00570551 0.99429449]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22419037 0.14809399 0.14436659 0.14631034 0.1445741  0.19246461]]
131
[[0.008349   0.991651  ]
 [0.00551401 0.99448599]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22440217 0.14820929 0.14446178 0.1456076  0.14468262 0.19263653]]
132
[[0.00809395 0.99190605]
 [0.00532968 0.99467032]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22417383 0.14809808 0.14437352 0.14631875 0.14457894 0.19245689]]
133
[[0.00784551 0.99215449]
 [0.00515082 0.99484918]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.2243876  0.14821398 0.14446906 0.14561115 0.14468798 0.19263022]]
134
[[0.00760586 0.99239414]
 [0.0

[[0.00233362 0.99766638]
 [0.00136686 0.99863314]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22399005 0.14813502 0.14444141 0.14644153 0.14462434 0.19236765]]
173
[[0.00226199 0.99773801]
 [0.00132108 0.99867892]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22424075 0.14826582 0.14454748 0.14563038 0.1447468  0.19256878]]
174
[[0.00219294 0.99780706]
 [0.00127703 0.99872297]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22398526 0.14813548 0.14444264 0.14644649 0.14462502 0.19236512]]
175
[[0.00212562 0.99787438]
 [0.00123426 0.99876574]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22423781 0.14826715 0.14454937 0.14562969 0.14474828 0.19256769]]
176
[[0.00206074 0.99793926]
 [0.00119312 0.99880688]]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.22398069 0.14813587 0.14444374 0.14645141 0.14462561 0.19236268]]
177
[[0.00199748 0.99800252]
 [0.0

In [206]:
model.B

array([[0.0763796 , 0.18127508, 0.2893702 , 0.17923852, 0.14367737,
        0.13005922],
       [0.27508139, 0.1317665 , 0.07349771, 0.12316039, 0.15347589,
        0.24301811]])

In [207]:
pd.Series(data).value_counts()

1    1298
6    1022
2     689
4     673
5     661
3     657
dtype: int64

In [76]:
x = np.array([1, 2])
print(x)
y = np.repeat(x[:, np.newaxis], len(x), axis=1)
print(y)

# y = np.repeat(x[:, np.newaxis], 2, axis=1)

# print(y)

[1 2]
[[1 1]
 [2 2]]


# OLD CODE

In [ ]:
def forward_proba(A, B, data):
    
    # get length of data 
    T = len(data)
    
    # initialize matrix to hold forward probabilities 
    alpha = np.zeros((num_hidden, T))
    
    # compute first time step 
    alpha[:,0] = np.multiply(pi, B[:,data[0]-1])
    
    # iterate over rest of time steps 
    for t in range(1, T): 
        alpha[:,t] = np.multiply(A @ alpha[:,t-1], B[:,data[t]-1])
    
    return alpha

In [ ]:
forward_probs = forward_proba(A, B, sample_data)
forward_probs

In [ ]:
def backward_proba(A, B, data): 
    
    # get length of data 
    T = len(data)
    
    # initialize matrix to hold backward probabilities 
    beta = np.zeros((num_hidden, T))
    
    # initialize last time step to 1 
    beta[:, -1] = 1 
    
    # iterate backward through rest of time steps 
    for t in reversed(range(T-1)):
        beta[:,t] = np.multiply(A @ beta[:,t+1], B[:,data[t]-1])
    
    return beta

In [ ]:
backward_probs = backward_proba(A, B, sample_data)

In [ ]:
backward_probs

In [ ]:
def compute_epsilon(A, B, alpha, beta, data): 
    
    # get length of data 
    T = len(data)

    # initialize epsilon matrix 
    epsilon = np.zeros((T, num_hidden, num_hidden))
    
    # loop from 1 to T to fill out the matrix 
    for t in range(T):
        epsilon[t,:,:] = 